In [1]:
!nvidia-smi

Fri Feb 27 12:47:50 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# --- CLEAN START ---
# Remove everything that can conflict
!pip uninstall -y unsloth unsloth-zoo transformers accelerate peft datasets tokenizers huggingface-hub bitsandbytes sentencepiece

# --- PYTORCH (CUDA 11.8) ---
!pip install torch --index-url https://download.pytorch.org/whl/cu118

# --- INSTALL MODERN UNSLOTH (the correct one for Qwen2.5) ---
!pip install "unsloth==2026.2.1"

# --- INSTALL THE HF STACK COMPATIBLE WITH UNSLOTH 2026.x ---
!pip install "transformers>=4.57.0" "accelerate>=0.34.1" "peft>=0.18.0" datasets sentencepiece bitsandbytes

# 1. Install Unsloth with the latest Colab-specific optimization
# !pip install --force-reinstall --no-cache-dir --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# 2. Install essential backends for LoRA and 4-bit quantization
# !pip install --no-deps "xformers<0.0.30" "trl<0.13.0" peft accelerate bitsandbytes

# 3. Install data handling and high-speed download utilities
# !pip install datasets sentencepiece hf_transfer huggingface_hub

# 4. Enable HF_TRANSFER for 10x faster model loading into Colab's local disk
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-9j101s15/unsloth_69e2dbfa0b254edf9ae3d73f56109e4d
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-9j101s15/unsloth_69e2dbfa0b254edf9ae3d73f56109e4d
  Resolved https://github.com/unslothai/unsloth.git to commit 96ea52f270dc62553ea2de598bf612207349b027
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2026.2.1-py3-none-any.whl size=451820 sha256=1d6fd1d3415db31f0ea1bb5297c23f2f941b6df34f891716e992fa10bf3cbd10
  Stored in directory: /tmp/pip-ephem-wheel-cache-tv5vttn_/wheels/60/3e/1f/e576c07051d90cf64b6a41434d87ccf4db33fafd5343bf5de0
Successfully built unsloth
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 21.7 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 32.3 MB/s eta 0:00:0

In [3]:
from google.colab import drive
import torch
import os
drive.mount('/content/drive')

with open('/content/drive/MyDrive/TMP/HF_TOKEN.txt') as f:
    token = f.read().strip()
    if token:
        os.environ["HF_TOKEN"] = token
# 2. Check if HF_TOKEN is set in the environment
test_token=os.getenv("HF_TOKEN")
if test_token:
    print("HF_TOKEN is set in the environment.")
else:
    print("HF_TOKEN is NOT set in the environment. Please check your token file and try again.")
    raise ValueError("HF_TOKEN not found in environment variables.")

# 2. Detect Hardware for optimization
major_v, _ = torch.cuda.get_device_capability()
IS_A100 = True if major_v >= 8 else False
print(f"🚀 Hardware: {'A100 (using BF16)' if IS_A100 else 'T4 (using FP16)'}")

Mounted at /content/drive
HF_TOKEN is set in the environment.
🚀 Hardware: T4 (using FP16)


In [4]:
import os

paths = {
    "Script": "/content/drive/MyDrive/scripts/train_scout_unsloth.py",
    "Data": "/content/drive/MyDrive/data/train/detective_finetune.jsonl"
}

for name, path in paths.items():
    if os.path.exists(path):
        print(f"✅ {name} found: {path}")
    else:
        print(f"❌ {name} MISSING: {path}")
        raise FileNotFoundError(f"{name} not found at {path}. Please check the path and try again.")

✅ Script found: /content/drive/MyDrive/scripts/train_scout_unsloth.py
✅ Data found: /content/drive/MyDrive/data/train/detective_finetune.jsonl


In [5]:
import time

script_path = "/content/drive/MyDrive/scripts/train_scout_unsloth.py"

print("🎬 Initializing fine-tuning session...")
start_time = time.time()

# Run the training script
!python {script_path}

end_time = time.time()
elapsed_mins = (end_time - start_time) / 60

print(f"\n" + "="*40)
print(f"🏁 TRAINING SESSION COMPLETE")
print(f"⏱️ Total Time: {elapsed_mins:.2f} minutes")
print(f"📂 Output Location: /content/drive/MyDrive/outputs/detective-qwen-sft")
print("="*40)

🎬 Initializing fine-tuning session...
Traceback (most recent call last):
  File "/usr/lib/python3.12/importlib/metadata/__init__.py", line 397, in from_name
    return next(cls.discover(name=name))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
StopIteration

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/unsloth/__init__.py", line 91, in <module>
    unsloth_zoo_version = importlib_version("unsloth_zoo")
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/importlib/metadata/__init__.py", line 889, in version
    return distribution(distribution_name).version
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/importlib/metadata/__init__.py", line 862, in distribution
    return Distribution.from_name(distribution_name)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/importlib/metadata/__init__.py", lin